In [2]:
import sys
!{sys.executable} -m pip install pdfrw

In [1]:
import json, urllib, csv
import os
import platform, subprocess

In [2]:
aliyot_file = open("source_files/aliyot.json",encoding='utf=8')
aliyot = json.load(aliyot_file)

In [3]:
def hebcal_haft_parser(haft_info):
    if type(haft_info) == list:
        if haft_info[0]["k"] == haft_info[1]["k"]:
            return haft_info[0]["k"]+" "+haft_info[0]["b"]+"-"+haft_info[0]["e"]+"; "+haft_info[1]["b"]+"-"+haft_info[1]["e"]
        else:
            return haft_info[0]["k"]+" "+haft_info[0]["b"]+"-"+haft_info[0]["e"]+"; "+haft_info[1]["k"]+" "+haft_info[1]["b"]+"-"+haft_info[1]["e"]
    else:
        return haft_info["k"]+" "+haft_info["b"]+"-"+haft_info["e"]

In [4]:
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word += letter
        if letter == "-" or letter == " " or letter == "־":
            no_cant_word += " "
    return no_cant_word

In [5]:
parsha_info = [{}]
with open('source_files/verse_numbers.csv') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    for row in csv_reader:
        parsha_data = {}
        parsha_data["name"] = row[1]
        parsha_data["num"] = row[0]
        parsha_data["versenum"] = row[2]
        for parsha in aliyot.values():
            if parsha["num"] == int(row[0]):
                haftarah_verses_string = "The Haftarah is "
                haftarah_verses_string += hebcal_haft_parser(parsha["haft"])
                haftarah_note = haftarah_verses_string + ". "
                haftarah_verses_string += r" on page \pageref{haft_"+row[0]+"}. "
                if "seph" in parsha.keys():
                    haftarah_verses_string += "Sepharadim read "+hebcal_haft_parser(parsha["seph"])+". "
                if row[3] != "":
                    haftarah_verses_string += row[3]
                    haftarah_note += row[3]
                haftarah_verses_string = haftarah_verses_string.replace("-",r"\verserangechar ")
                haftarah_verses_string = haftarah_verses_string.replace("; ", r" \tworange ")
                haftarah_note = haftarah_note.replace("-",r"\verserangechar ")
                haftarah_note = haftarah_note.replace("; ", r" \tworange ")
                parsha_data["haft_note"] = haftarah_verses_string
                parsha_data["haft_note_2"] = haftarah_note
                parsha_data["he_name"] = nonalpha_remover(parsha["hebrew"])
        parsha_info.append(parsha_data)

In [6]:
weekday_aliya_names = [None, None,"לוי","ישראל","ע״כ בחול"]
shabbat_aliya_names = [None,None,"שני","שלישי","רביעי","חמישי","ששי","שביעי","מפטיר"]
books = ["","Genesis","Exodus","Leviticus","Numbers","Deuteronomy"]

In [7]:
sedarim = []
with open('source_files/sedarim_list.csv',encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    for row in csv_reader:
        seder = {}
        seder["num"] = row[1]
        seder["num_en"] = row[0]
        seder["book"] = row[2]
        seder["bk_num"] = books.index(row[2])
        seder["chap"] = int(row[3])
        seder["verse"] = int(row[4])
        sedarim.append(seder)

In [8]:
def qkparse(string):
    start = string.index('<span class=\"mam-kq">')
    string_before = string[0:start]
    startk = string.index('<span class=\"mam-kq-k\">',start)
    startk = string.index('>',startk)+2
    endk = string.index('</span>',startk)-1
    ktext = string[startk:endk]
    startq = string.index('<span class=\"mam-kq-q\">')
    startq = string.index('>',startq)+2
    endq = string.index('</span>',startq)-1
    qtext=string[startq:endq]
    end = string.index('</span></span>',endq)
    if " " in string[end:]:
        end = string.index(' ',end)
    else:
        end = string.index('׃',end)
    output = '\\qk{'+qtext+'}{'+ktext+'}'
    string_after = string[end:]
    return string_before+output+string_after

In [9]:
def removeformatting(text):
    text = text.replace("<span class=\"mam-kq-k\">(נא)",r"(\textit{כתיב ולא קרי} נא)")
    while '<span class=\"mam-kq\">' in text:
        text = qkparse(text)
    text = text.replace("{פ}",r"\petucha ")
    text = text.replace("{ס}",r"\setuma ")
    text = text.replace("<span>","")
    text = text.replace("</span>","")
    text = text.replace("<small>׀</small>",r"\pasek ")
    text = text.replace("<b>׀</b>",r"\legarmeh ")
    text = text.replace("־",r"\maqqaf ")
    text = text.replace("<small>",r"\footnotesize ")
    text = text.replace("</small> ",r"\normalsize\space ")
    text = text.replace("</small>",r"\normalsize ")
    text = text.replace("<br>","")
    text = text.replace("<span class=\"mam-kq-trivial\">","")
    text = text.replace("<big>",r"\large ")
    text = text.replace("</big> ",r"\normalsize\space ")
    text = text.replace("</big>",r"\normalsize ")
    text = text.replace("<sup>*</sup><i class=\"footnote\">(",r"\note{")
    text = text.replace(')</i>',r"}")
    text = text.replace("           "," ")
    text = text.replace("         "," ")
    text = text.replace("          "," ")
    text = text.replace("<span class=\"mam-spi-invnun\">","")
    if "<" in text:
        print(text)
    return text

In [10]:
def parserashi(content, chap, vs):
    rashi_chap = content["text"][chap-1]
    if len(rashi_chap)<vs:
        return ""
    rashi_list = rashi_chap[vs-1]
    rashi_text = r"\rashi{"
    for comment in rashi_list:
        if comment == "":
            continue
        comment = comment.replace("{","")
        comment = comment.replace("}","")
        comment = comment.replace("<b>",r"\rashiDH{")
        comment = comment.replace("</b>",r"}")
        comment = comment.replace("<small>","")
        comment = comment.replace("</small>","")
        comment = comment.replace(r"\"","״")
        comment = comment.replace("\"","״")
        comment = comment.replace(r"'","׳")
        comment = comment.replace(r":","׃")
        comment = comment.replace(r"—","")
        comment = comment.replace(r"<i data-commentator=״Siftei Chakhamim״ data-label=״⚬״></i>","")
        rashi_text += comment + r"\quad "
    rashi_text += "}"
    rashi_text = rashi_text.replace(r"\quad }","}")
    rashi_text = rashi_text.replace(r"\rashi{\rashiDH{ואם בבהמה הטמאה וגו׳.} }",r"\rashi{\rashiDH{ואם בבהמה הטמאה וגו׳.}")
    if rashi_text == r"\rashi{}":
        return None
    else:
        return rashi_text

In [11]:
def parseenglish(text):
    text = text.replace(r"s’",r"s’ ")
    text = text.replace("LORD",r"\lord")
    text = text.replace(r"\lord ",r"\lord\space ")
    text = text.replace("I AM THAT I AM",r"\textsc{I Am That I Am}")
    text = text.replace("I AM",r"\textsc{I Am}")
    text = text.replace("יהוה",r"\englishshem")
    text = text.replace("GOD",r"\textsc{God}")
    return text

In [12]:
def parsealiyot(aliyot,book_num,vs_id):
    aliyot = dict(reversed(list(aliyot.items())))
    combo = None
    for parsha in aliyot.values():
        if parsha["book"] != book_num:
            continue
        elif parsha["fullkriyah"]["1"]["b"] == vs_id and "combined" not in parsha.keys():
            return ["parsha",nonalpha_remover(parsha["hebrew"])]
        elif parsha["fullkriyah"]["7"]["e"] == vs_id and "combined" not in parsha.keys():
            return ["parsha","end",parsha["num"]]
        elif "combined" in parsha.keys():
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    combo = aliya_number
                aliya_number += 1
        else:
            aliya_number = 1
            for aliya in parsha["fullkriyah"].values():
                if aliya["b"] == vs_id:
                    if combo == None or aliya_number == 8:
                        return ["aliya",aliya_number]
                    else:
                        return ["combo",[aliya_number, combo]]
                aliya_number += 1
            aliya_number = 1
            if "weekday" in parsha.keys():
                for weekday in parsha["weekday"].values():
                    if weekday["b"]==vs_id:
                        return ["weekday",aliya_number]
                    elif vs_id == parsha["weekday"]["3"]["e"] and vs_id != parsha["fullkriyah"]["1"]["e"]:
                        return ["weekday",4]
                    aliya_number += 1
    return

In [13]:
taam_tachton_ex = ["","","אָֽנֹכִ֖י יְהֹוָ֣ה אֱלֹהֶ֑יךָ אֲשֶׁ֧ר הוֹצֵאתִ֛יךָ מֵאֶ֥רֶץ מִצְרַ֖יִם מִבֵּ֥ית עֲבָדִֽים׃",
"לֹֽא־יִהְיֶ֥ה לְךָ֛ אֱלֹהִ֥ים אֲחֵרִ֖ים עַל־פָּנָֽי׃",
"לֹֽא־תַעֲשֶׂ֨ה לְךָ֥ פֶ֙סֶל֙ וְכׇל־תְּמוּנָ֔ה אֲשֶׁ֤ר בַּשָּׁמַ֙יִם֙ מִמַּ֔עַל וַֽאֲשֶׁ֥ר בָּאָ֖רֶץ מִתָּ֑חַת וַאֲשֶׁ֥ר בַּמַּ֖יִם מִתַּ֥חַת לָאָֽרֶץ׃",
"לֹֽא־תִשְׁתַּחֲוֶ֥ה לָהֶ֖ם וְלֹ֣א תׇעׇבְדֵ֑ם כִּ֣י אָֽנֹכִ֞י יְהֹוָ֤ה אֱלֹהֶ֙יךָ֙ אֵ֣ל קַנָּ֔א פֹּ֠קֵ֠ד עֲוֺ֨ן אָבֹ֧ת עַל־בָּנִ֛ים עַל־שִׁלֵּשִׁ֥ים וְעַל־רִבֵּעִ֖ים לְשֹׂנְאָֽי׃",
"וְעֹ֥שֶׂה חֶ֖סֶד לַאֲלָפִ֑ים לְאֹהֲבַ֖י וּלְשֹׁמְרֵ֥י מִצְוֺתָֽי׃"+r"\setuma",
"לֹ֥א תִשָּׂ֛א אֶת־שֵֽׁם־יְהֹוָ֥ה אֱלֹהֶ֖יךָ לַשָּׁ֑וְא כִּ֣י לֹ֤א יְנַקֶּה֙ יְהֹוָ֔ה אֵ֛ת אֲשֶׁר־יִשָּׂ֥א אֶת־שְׁמ֖וֹ לַשָּֽׁוְא׃"+r"\petucha",
"זָכ֛וֹר אֶת־י֥וֹם הַשַּׁבָּ֖ת לְקַדְּשֽׁוֹ׃",
"שֵׁ֤שֶׁת יָמִים֙ תַּֽעֲבֹ֔ד וְעָשִׂ֖יתָ כׇּל־מְלַאכְתֶּֽךָ׃",
"וְיוֹם֙ הַשְּׁבִיעִ֔י שַׁבָּ֖ת לַיהֹוָ֣ה אֱלֹהֶ֑יךָ לֹֽא־תַעֲשֶׂ֨ה כׇל־מְלָאכָ֜ה אַתָּ֣ה ׀ וּבִנְךָ֣ וּבִתֶּ֗ךָ עַבְדְּךָ֤ וַאֲמָֽתְךָ֙ וּבְהֶמְתֶּ֔ךָ וְגֵרְךָ֖ אֲשֶׁ֥ר בִּשְׁעָרֶֽיךָ׃",
"כִּ֣י שֵֽׁשֶׁת־יָמִים֩ עָשָׂ֨ה יְהֹוָ֜ה אֶת־הַשָּׁמַ֣יִם וְאֶת־הָאָ֗רֶץ אֶת־הַיָּם֙ וְאֶת־כׇּל־אֲשֶׁר־בָּ֔ם וַיָּ֖נַח בַּיּ֣וֹם הַשְּׁבִיעִ֑י עַל־כֵּ֗ן בֵּרַ֧ךְ יְהֹוָ֛ה אֶת־י֥וֹם הַשַּׁבָּ֖ת וַֽיְקַדְּשֵֽׁהוּ׃"+r"\setuma",
"כַּבֵּ֥ד אֶת־אָבִ֖יךָ וְאֶת־אִמֶּ֑ךָ לְמַ֙עַן֙ יַאֲרִכ֣וּן יָמֶ֔יךָ עַ֚ל הָאֲדָמָ֔ה אֲשֶׁר־יְהֹוָ֥ה אֱלֹהֶ֖יךָ נֹתֵ֥ן לָֽךְ׃"+r"\setuma",
r"לֹ֥א תִרְצָ֖ח \setuma לֹ֣א תִנְאָ֑ף \setuma לֹ֣א תִגְנֹ֔ב לֹֽא־תַעֲנֶ֥ה בְרֵעֲךָ֖ עֵ֥ד שָֽׁקֶר׃ \setuma",
"לֹ֥א תַחְמֹ֖ד בֵּ֣ית רֵעֶ֑ךָ לֹֽא־תַחְמֹ֞ד אֵ֣שֶׁת רֵעֶ֗ךָ וְעַבְדּ֤וֹ וַאֲמָתוֹ֙ וְשׁוֹר֣וֹ וַחֲמֹר֔וֹ וְכֹ֖ל אֲשֶׁ֥ר לְרֵעֶֽךָ׃"+r"\petucha"]

In [14]:
taam_tachton_deut = ["","","","","","","אָֽנֹכִ֖י יְהֹוָ֣ה אֱלֹהֶ֑יךָ אֲשֶׁ֧ר הוֹצֵאתִ֛יךָ מֵאֶ֥רֶץ מִצְרַ֖יִם מִבֵּ֥ית עֲבָדִֽים׃",
                     "לֹא־יִהְיֶ֥ה לְךָ֛ אֱלֹהִ֥ים אֲחֵרִ֖ים עַל־פָּנָֽי׃",
                     "לֹֽא־תַעֲשֶׂ֨ה לְךָ֥ פֶ֙סֶל֙ כׇּל־תְּמוּנָ֔ה אֲשֶׁ֤ר בַּשָּׁמַ֙יִם֙ מִמַּ֔עַל וַאֲשֶׁ֥ר בָּאָ֖רֶץ מִתָּ֑חַת וַאֲשֶׁ֥ר בַּמַּ֖יִם מִתַּ֥חַת לָאָֽרֶץ׃",
                     "לֹא־תִשְׁתַּחֲוֶ֥ה לָהֶ֖ם וְלֹ֣א תׇעׇבְדֵ֑ם כִּ֣י אָנֹכִ֞י יְהֹוָ֤ה אֱלֹהֶ֙יךָ֙ אֵ֣ל קַנָּ֔א פֹּ֠קֵ֠ד עֲוֺ֨ן אָב֧וֹת עַל־בָּנִ֛ים וְעַל־שִׁלֵּשִׁ֥ים וְעַל־רִבֵּעִ֖ים לְשֹׂנְאָֽי׃",
                     r"וְעֹ֥שֶׂה חֶ֖סֶד לַֽאֲלָפִ֑ים לְאֹהֲבַ֖י וּלְשֹׁמְרֵ֥י \qk{מִצְוֺתָֽי}{מצותו}׃"+r"\setuma",
                     "לֹ֥א תִשָּׂ֛א אֶת־שֵֽׁם־יְהֹוָ֥ה אֱלֹהֶ֖יךָ לַשָּׁ֑וְא כִּ֣י לֹ֤א יְנַקֶּה֙ יְהֹוָ֔ה אֵ֛ת אֲשֶׁר־יִשָּׂ֥א אֶת־שְׁמ֖וֹ לַשָּֽׁוְא׃"+r"\setuma",
                    "שָׁמ֛וֹר אֶת־י֥וֹם הַשַּׁבָּ֖ת לְקַדְּשׁ֑וֹ כַּאֲשֶׁ֥ר צִוְּךָ֖ יְהֹוָ֥ה אֱלֹהֶֽיךָ׃",
                     "שֵׁ֤שֶׁת יָמִים֙ תַּֽעֲבֹ֔ד וְעָשִׂ֖יתָ כׇּל־מְלַאכְתֶּֽךָ׃",
                     "וְיוֹם֙ הַשְּׁבִיעִ֔י שַׁבָּ֖ת לַיהֹוָ֣ה אֱלֹהֶ֑יךָ לֹ֣א תַעֲשֶׂ֣ה כׇל־מְלָאכָ֡ה אַתָּ֣ה וּבִנְךָֽ־וּבִתֶּ֣ךָ וְעַבְדְּךָֽ־וַ֠אֲמָתֶ֠ךָ וְשׁוֹרְךָ֨ וַחֲמֹֽרְךָ֜ וְכׇל־בְּהֶמְתֶּ֗ךָ וְגֵֽרְךָ֙ אֲשֶׁ֣ר בִּשְׁעָרֶ֔יךָ לְמַ֗עַן יָנ֛וּחַ עַבְדְּךָ֥ וַאֲמָתְךָ֖ כָּמֽוֹךָ׃",
                     "וְזָכַרְתָּ֗ כִּ֣י עֶ֤בֶד הָיִ֙יתָ֙ בְּאֶ֣רֶץ מִצְרַ֔יִם וַיֹּצִ֨אֲךָ֜ יְהֹוָ֤ה אֱלֹהֶ֙יךָ֙ מִשָּׁ֔ם בְּיָ֥ד חֲזָקָ֖ה וּבִזְרֹ֣עַ נְטוּיָ֑ה עַל־כֵּ֗ן צִוְּךָ֙ יְהֹוָ֣ה אֱלֹהֶ֔יךָ לַעֲשׂ֖וֹת אֶת־י֥וֹם הַשַּׁבָּֽת׃"+r"\setuma",
                     "כַּבֵּ֤ד אֶת־אָבִ֙יךָ֙ וְאֶת־אִמֶּ֔ךָ כַּאֲשֶׁ֥ר צִוְּךָ֖ יְהֹוָ֣ה אֱלֹהֶ֑יךָ לְמַ֣עַן ׀ יַאֲרִיכֻ֣ן יָמֶ֗יךָ וּלְמַ֙עַן֙ יִ֣יטַב לָ֔ךְ עַ֚ל הָֽאֲדָמָ֔ה אֲשֶׁר־יְהֹוָ֥ה אֱלֹהֶ֖יךָ נֹתֵ֥ן לָֽךְ׃"+r"\setuma",
                     "לֹ֥א תִרְצַ֖ח \setuma וְלֹ֣א תִנְאָ֑ף \setuma וְלֹ֣א תִגְנֹ֔ב \setuma וְלֹֽא־תַעֲנֶ֥ה בְרֵֽעֲךָ֖ עֵ֥ד שָֽׁוְא׃"+r"\setuma",
                     "וְלֹ֥א תַחְמֹ֖ד אֵ֣שֶׁת רֵעֶ֑ךָ וְלֹ֨א תִתְאַוֶּ֜ה בֵּ֣ית רֵעֶ֗ךָ שָׂדֵ֜הוּ וְעַבְדּ֤וֹ וַאֲמָתוֹ֙ שׁוֹר֣וֹ וַחֲמֹר֔וֹ וְכֹ֖ל אֲשֶׁ֥ר לְרֵעֶֽךָ׃"+r"\setuma"]

In [15]:
def specialcase(heb, book, chap, verse):
    if book == 2 and chap == 20 and 1 < verse < 15:
        heb = taam_tachton_ex[verse]
    elif book == 2 and chap == 15 and verse < 20:
        heb = heb.replace(r"\setuma","")
    elif book == 5 and chap == 32 and verse < 44:
        heb = heb.replace(r"\setuma","")
    return heb

In [16]:
def addnotes(booknum, chapnum, vsnum):
    note_text = None
    if booknum == 2 and chapnum==20 and vsnum==2:
        note_text = r"The Ten Commandments are presented here in Ta'am Ta\d{h}ton. For the Ten Commandments in Ta'am Elyon, see page \pageref{elyon}."
    elif booknum == 2 and chapnum == 15 and vsnum==1:
        note_text = r"For brevity, all notes of breaks in the Song of the Sea are omitted. For the proper formatting for this passage, see page \pageref{shirathayam}."
    elif booknum == 5 and chapnum == 5 and vsnum == 6:
        note_text = r"The Ten Commandments are presented here in Ta'am Ta\d{h}ton. For the Ten Commandments in Ta'am Elyon, see page \pageref{elyon}."
    elif booknum == 5 and chapnum == 32 and vsnum == 1:
        note_text = r"For brevity, the layout of the Song of Moses is not noted here. For the proper formatting for this passage, see page \pageref{haazinu}."
    return note_text

In [17]:
special_maftirs = []
special_maftirs.append({'name':'hachodesh','book':2,'chap':12,'vs':1})
special_maftirs.append({'name':'chanukkah','book':4,'chap':12,'vs':1})
special_maftirs.append({'name':'parah','book':4,'chap':19,'vs':1})
special_maftirs.append({'name':'shekalim','book':2,'chap':30,'vs':11})
special_maftirs.append({'name':'hachodesh','book':2,'chap':12,'vs':1})
special_maftirs.append({'name':'zachor','book':5,'chap':25,'vs':17})
special_maftirs.append({'name':'ShabRCh','book':4,'chap':28,'vs':9})

In [18]:
def addlabels(booknum,chap,vs):
    label_text = None
    for maftir in special_maftirs:
        if booknum == maftir['book'] and chap == maftir['chap'] and vs == maftir['vs']:
            label_text = r"\label{maft_"+maftir['name']+"}"
            break
    return label_text

In [19]:
def TeXwriter(outputfile,content):
    with open(outputfile, 'w', encoding='utf-8') as outfile:
        for line in content:
            outfile.write(line)
            outfile.write("\n")

In [20]:
def parsetarg(vs):
    vs = vs.replace("{","")
    vs = vs.replace("}","")
    return vs

In [21]:
def make_chumash(heb, targ, eng, rashi, parsha_data, book):
    versecounter = 1
    chapcounter = 1
    output = []
    for chapter in heb["text"]:
        for verse in chapter:
            id_str = str(chapcounter)+":"+str(versecounter)
            verse_he = removeformatting(verse)
            verse_he = specialcase(verse_he,book,chapcounter,versecounter)
            heb_id = parsealiyot(parsha_data, book, id_str)
            engnote = ""
            if heb_id == None:
                if versecounter == 1:
                    versetext_he = r'\Roman{chap}'
                else:
                    versetext_he = r"\arabic{verse}"
            elif heb_id[0] == "parsha" and heb_id[1]!= "end":
                output.append(r"\newparsha{"+heb_id[1]+"}")
                versetext_he = r"\aliya{"+heb_id[1]+"}"
            elif heb_id[0]=="parsha" and heb_id[1] == "end":
                parsha_num = heb_id[2]
                verses = parsha_info[parsha_num]["versenum"]
                haft_note = parsha_info[parsha_num]["haft_note"]
                parsha_name = parsha_info[parsha_num]["he_name"]
                versetext_he = r"\aliya{\hebrewnumeral{"+verses+r"}}\newline\aliya{פסוקים}"
                engnote = r"\engnote{"+haft_note+"}"
            elif heb_id[0] == "weekday":
                aliya_text = weekday_aliya_names[heb_id[1]]
                versetext_he = r"\aliya{"+aliya_text+"}"
            elif heb_id[0] == "combo":
                aliya_text = [shabbat_aliya_names[heb_id[1][1]],shabbat_aliya_names[heb_id[1][0]]]
                versetext_he = r"\aliya{"+aliya_text[1]+r"\newline ("+aliya_text[0]+")}"
            elif heb_id[0] == "aliya":
                aliya_text = shabbat_aliya_names[heb_id[1]]
                versetext_he = r"\aliya{"+aliya_text+"}"
            if versecounter == 1:
                versetext_en = r'\Roman{chap}'
            else:
                versetext_en = r"\arabic{verse}"
            seder_note = ""
            for seder in sedarim:
                if seder['bk_num']==book and seder['chap']==chapcounter and seder['verse']==versecounter:
                    seder_note = seder['num']
                    if "*" not in seder_note and "(" not in seder_note:
                        output.append(r"\newseder{"+seder['num_en']+"}")
                    if "aliya" in versetext_he:
                        versetext_he+=r"\newline\vspace{-4pt}\newline\seder{"+seder_note+"}"
                    else:
                        versetext_he = r"\seder{"+seder_note+"}"
            rashi_pasuk = parserashi(rashi,chapcounter,versecounter)
            targ_vs = parsetarg(targ["text"][chapcounter-1][versecounter-1])
            english = parseenglish(eng["text"][chapcounter-1][versecounter-1])
            english_notes = addnotes(book,chapcounter,versecounter)
            label = addlabels(book,chapcounter,versecounter)
            if label != None:
                output.append(label)
            if english_notes != None:
                output.append(r"\engnote{"+english_notes+r"}")
            output.append(r'\threeverse{'+versetext_he+"}%"+books[book]+str(chapcounter)+r':'+str(versecounter))
            output.append(r'{'+verse_he+'}')
            output.append(r"{"+targ_vs+r"}")
            output.append("{"+english+r"}{"+versetext_en+r"}")
            if rashi_pasuk != None:
                output.append(rashi_pasuk)
            if engnote != "":
                output.append(engnote)
            versecounter += 1
        chapcounter += 1
        output.append(r"\newperek")
        versecounter = 1
    #output[-1] = r"}"
    return output

In [32]:
def haftarahpuller(haftarah_name, haft_label, text_ref, haft_note):
    text_ref = text_ref.replace(" ","_")
    link = "https://www.sefaria.org/api/texts/"+text_ref+"?ven=The_Holy_Scriptures:_A_New_Translation_(JPS_1917)&vhe=Miqra_according_to_the_Masorah&context=0"
    print(link)
    with urllib.request.urlopen(link) as url:
        sefariaresult = json.loads(url.read().decode())
    output = []
    if haftarah_name != None:
        output.append(r"\newhaftarah{"+haftarah_name+"}")
        output.append(r"\label{"+haft_label+"}")
        output.append(r"\haftnote{"+haft_note+"}")
    #english_start_string = sefariaresult["book"]+r" \Roman{chap}:\arabic{verse} \textendash "
    #english_end_string = r"\romannumeral "+str(sefariaresult["toSections"][0])+" :"+str(sefariaresult["toSections"][1])
    english_string = sefariaresult["ref"]
    english_string = english_string.replace("-",r"\verserangechar ")
    hebrew_string = r"\textsf{"+sefariaresult["heRef"]+r"}"
    #output.append(r"\twoverse{}{"+hebrew_string+"}{"+english_string+"}")
    output.append(r"\setcounter{chap}{"+str(sefariaresult["sections"][0])+"}")
    output.append(r"\setcounter{verse}{"+str(sefariaresult["sections"][1])+"}")
    if type(sefariaresult["he"])==str:
        text_he = removeformatting(sefariaresult["he"])
        text_en = parseenglish(sefariaresult["text"])
        output.append(r"\twoverse{\Roman{chap}:\arabic{verse}}")
        output.append("{"+text_he+"}")
        output.append("{"+text_en+"}")
    elif type(sefariaresult["he"])==list:
        if type(sefariaresult["he"][0])==str:
            sef_id = 0
            for verse in sefariaresult["he"]:
                text_he = removeformatting(verse)
                text_en = parseenglish(sefariaresult["text"][sef_id])
                if sef_id == 0:
                    id_text = r"\Roman{chap}:\arabic{verse}"
                else:
                    id_text = r"\arabic{verse}"
                output.append(r"\twoverse{"+id_text+"}")
                output.append("{"+text_he+"}")
                output.append("{"+text_en+"}")
                sef_id += 1
        elif type(sefariaresult["he"][0])==list:
            sef_id = [0,0]
            for chapter in sefariaresult["he"]:
                for verse in chapter:
                    text_he = removeformatting(verse)
                    text_en = parseenglish(sefariaresult["text"][sef_id[0]][sef_id[1]])
                    if sef_id[1] == 0:
                        id_text = r"\Roman{chap}:\arabic{verse}"
                    else:
                        id_text = r"\arabic{verse}"
                    output.append(r"\twoverse{"+id_text+"}")
                    output.append("{"+text_he+"}")
                    output.append("{"+text_en+"}")
                    sef_id[1] += 1
                output.append(r"\newperek")
                sef_id[0] += 1
                sef_id[1] = 0
    

    return output

In [33]:
def get_haftarot(aliyot, book):
    output = [r"\vspace{24pt}\LARGE \textbf{הפטרות} \normalsize\vspace{-16pt}",r"\addcontentsline{toc}{chapter}{הפטרות}"]
    for parsha in aliyot.values():
        if parsha["book"] == book and "haft" in parsha.keys():
            haftara_pesukim = parsha["haft"]
            haftara_name = nonalpha_remover(parsha["hebrew"])
            #haft_label = "haft_"+nonalpha_remover(parsha)
            haft_label = "haft_"+str(parsha["num"])
            haft_note = parsha_info[parsha["num"]]["haft_note_2"]
            haft_strings = []
            if type(haftara_pesukim) == list:
                for chunk in haftara_pesukim:
                    string = chunk["k"]+"."+chunk["b"]+"-"+chunk["e"]
                    haft_strings.append(string)
            else:
                string = haftara_pesukim["k"]+"."+haftara_pesukim["b"]+"-"+haftara_pesukim["e"]
                haft_strings = [string]
            for chunk in haft_strings:
                output += haftarahpuller(haftara_name, haft_label, chunk, haft_note)
                haftara_name = None
    return output

In [24]:
#haftarot = get_haftarot(aliyot,2)
#TeXwriter("Chumash_files/2exodus_haftarot.tex",haftarot)

In [36]:
# special = haftarahpuller("שבת ראש חודש","maft_ShabRCh","Numbers 28:9 - 28:15")
# special += haftarahpuller("שבת ראש חודש","haft_ShabRCh","Isaiah 66:1 - 66:24")
# special += haftarahpuller("שבת מחר חודש","haft_macharchodesh","I Samuel 20:18 - 20:42")
# special += haftarahpuller("פרשת שקלים","haft_shekalim","II Kings 12:1 - 12:17")
# special += haftarahpuller("פרשת זכור","maft_zachor","Deuteronomy 25:17 - 25:19")
# special += haftarahpuller("פרשת זכור","haft_zachor","I Samuel 15:2 - 15:34")
# special += haftarahpuller("פרשת פרה","maft_parah","Numbers 19:1 - 19:22")
# special += haftarahpuller("פרשת פרה","haft_parah","Ezekiel 36:16 - 36:38")
#special = haftarahpuller("פרשת החודש","maft_hachodesh","Exodus 12:1 - 12:20")
# special += haftarahpuller("פרשת החודש","haft_hachodesh","Ezekiel 45:16 - 46:18")
#special += haftarahpuller("שבת הגדול", "haft_hagadol", "Malachi 3:4 - 3:24")
special = haftarahpuller("חנכה","maft_hanukka","Numbers_7:1-8:4","")
for line in special:
    print(line)



https://www.sefaria.org/api/texts/Numbers_7:1-8:4?ven=The_Holy_Scriptures:_A_New_Translation_(JPS_1917)&vhe=Miqra_according_to_the_Masorah&context=0
\newhaftarah{חנכה}
\label{maft_hanukka}
\haftnote{}
\setcounter{chap}{7}
\setcounter{verse}{1}
\twoverse{\Roman{chap}:\arabic{verse}}
{וַיְהִ֡י בְּיוֹם֩ כַּלּ֨וֹת מֹשֶׁ֜ה לְהָקִ֣ים אֶת\maqqaf הַמִּשְׁכָּ֗ן וַיִּמְשַׁ֨ח אֹת֜וֹ וַיְקַדֵּ֤שׁ אֹתוֹ֙ וְאֶת\maqqaf כׇּל\maqqaf כֵּלָ֔יו וְאֶת\maqqaf הַמִּזְבֵּ֖חַ וְאֶת\maqqaf כׇּל\maqqaf כֵּלָ֑יו וַיִּמְשָׁחֵ֖ם וַיְקַדֵּ֥שׁ אֹתָֽם׃}
{And it came to pass on the day that Moses had made an end of setting up the tabernacle, and had anointed it and sanctified it, and all the furniture thereof, and the altar and all the vessels thereof, and had anointed them and sanctified them;}
\twoverse{\arabic{verse}}
{וַיַּקְרִ֙יבוּ֙ נְשִׂיאֵ֣י יִשְׂרָאֵ֔ל רָאשֵׁ֖י בֵּ֣ית אֲבֹתָ֑ם הֵ֚ם נְשִׂיאֵ֣י הַמַּטֹּ֔ת הֵ֥ם הָעֹמְדִ֖ים עַל\maqqaf הַפְּקֻדִֽים׃}
{that the princes of Israel, the heads of their fathers’ houses, o

In [26]:
def pull_chumash_data(book, chumash_writebool,haft_writebool):
    bookname = books[book]   
    torah_file = open("source_files/Torah/"+str(book)+bookname+"_MAPM.json",encoding='utf=8')
    MAPM = json.load(torah_file)
    english_file = open("source_files/English/"+str(book)+bookname+"_JPS.json",encoding='utf=8')
    JPS = json.load(english_file)
    targum_file = open("source_files/Targum/"+str(book)+bookname+"_Targum.json",encoding='utf=8')
    targ = json.load(targum_file)
    rashi_file = open("source_files/Rashi/"+str(book)+bookname+"_Rashi.json",encoding='utf=8')
    rashi = json.load(rashi_file)
    result = make_chumash(MAPM, targ, JPS, rashi, aliyot, book)
    
    if chumash_writebool == True:
        TeXwriter("Chumash_files/"+str(book)+bookname+"_chumash.tex",result)
    if haft_writebool == True:
        haftarot = get_haftarot(aliyot,book)
        TeXwriter("Chumash_files/"+str(book)+bookname+"_haftarot.tex",haftarot)
    else:
        haftarot = None
    return result, haftarot

In [27]:
from pdfrw import PdfReader, PdfWriter

def flip_PDF(inpfn):
    rotate = 180

    #ranges = [[int(y) for y in x.split('-')] for x in ranges]
    outname = inpfn.split(".")
    outfn = outname[0]+".rotated."+outname[1]
    #outfn = 'rotate.%s' % os.path.basename(inpfn)
    trailer = PdfReader(inpfn)
    pages = trailer.pages

    ranges = [[1, len(pages)]]

    for onerange in ranges:
        onerange = (onerange + onerange[-1:])[:2]
        for pagenum in range(onerange[0]-1, onerange[1]):
            pages[pagenum].Rotate = (int(pages[pagenum].inheritable.Rotate or
                                         0) + rotate) % 360

    outdata = PdfWriter(outfn)
    outdata.trailer = trailer
    outdata.write()

In [40]:
for i in range(1,2):
    pull_chumash_data(i,True,False)
    filename = str(i)+"_"+books[i]+".tex"
    print(books[i]+" tex file generated")
    subprocess.run(['xelatex', '-interaction=nonstopmode', filename])
    print(books[i]+" first pass complete")
    subprocess.run(['xelatex', '-interaction=nonstopmode', filename])
    print(books[i]+" second pass complete")
    #compile twice to make sure all the labels are correct
    file_pdf = filename.replace("tex","pdf")
    flip_PDF(file_pdf)
    print(books[i]+" PDF flipped")

Genesis tex file generated
Genesis first pass complete
Genesis second pass complete
Genesis PDF flipped


In [29]:
flip_PDF("covers/3Leviticus_cover.pdf")
flip_PDF("covers/4Numbers_cover.pdf")